In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pingouin as pg
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from os.path import join as pjoin
from scipy.stats import zscore, norm
import sys
import re
from tqdm import tqdm

In [3]:
sys.path.append('..')
import circletrack_behavior as ctb
import plotting_functions as pf
import pca_ica as ica 

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [4]:
## Create a list of sessions for easy plotting later and for changing column order
session_list = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']
## Create session_id dictionary
session_dict = {'mc03': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal4', 'Training_Reversal'],
                'mc06': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc07': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc09': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc11': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']}
## Set path variables
ensemble_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/assemblies'
behavior_path = '../../EnsembleRemodeling_Resubmission/circletrack_data'   
figure_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/python_analysis/progress_figures/cohort1'   

In [ ]:
## Set relative path variable for circletrack behavior data
path = '../Data/**/**/**/circle_track.csv'
## Set str2match variable (regex for mouse name)
str2match = '(mc[0-9]+)'

In [ ]:
## Create list of files
file_list = ctb.get_file_list(path)

In [ ]:
## Loop through file_list to extract mouse name
mouseID = []
for file in file_list:
    mouse = ctb.get_mouse(file, str2match)
    mouseID.append(mouse)
## Combine file_list and mouseID
combined_list = ctb.combine(file_list, mouseID)

In [ ]:
## Loop through mice to get first lick accuracy
lick_accuracy = pd.DataFrame()
for mouse in combined_list.mouse.unique():
    flhr = ctb.get_lick_accuracy(combined_list, mouse)
    lick_accuracy = pd.concat([lick_accuracy, flhr], ignore_index = True)

In [ ]:
## Plot first lick accuracy
fig = pf.plot_behavior_across_days(lick_accuracy, x_var = 'day', y_var = 'percent_correct', avg_color = 'turquoise')
fig.update_layout(yaxis_title = 'Percent Correct (%)')
fig.update_layout(title = {'text': 'Lick Bout Accuracy',
                                   'xanchor': 'center',
                                   'y': 0.9,
                                   'x': 0.5})
fig.add_hline(y = 75, line_dash = 'dash', line_width = 1, line_color = 'black')
fig.update_layout(width = 500, height = 500)
fig.update_yaxes(range = [0, 90])
fig.show()
# fig.write_image('progress_figures/lick_bout_accuracy.png')

In [ ]:
## Loop through mice to get total rewards
reward_data = pd.DataFrame()
for mouse in combined_list.mouse.unique():
    reward = ctb.get_total_rewards(combined_list, mouse)
    reward_data = pd.concat([reward_data, reward], ignore_index = True)

In [ ]:
## Plot figure
fig = pf.plot_behavior_across_days(reward_data, x_var = 'day', y_var = 'total_rewards', avg_color = 'turquoise')
fig.update_layout(yaxis_title = 'Total Rewards')
fig.update_layout(title = {'text': 'Rewards per Day',
                                   'xanchor': 'center',
                                   'y': 0.9,
                                   'x': 0.5})
fig.update_layout(width = 500, height = 500)
fig.show()
# fig.write_image('progress_figures/rewards.png')

In [ ]:
## Correct direction proportion
direction_percentage = pd.DataFrame()
for mouse in combined_list.mouse.unique():
    ## Get location data of mouse
    location = ctb.get_location_data(combined_list, mouse)
    ## Determine if mouse was in correct or incorrect direction
    location = ctb.get_direction_information(location, lagn = 15)
    direction = ctb.direction_percentage(location)
    direction_percentage = pd.concat([direction_percentage, direction], ignore_index = True)

In [ ]:
for i in range(0, 4):
    location.loc[i, 'timestamp'] = location.loc[i+1, 'timestamp']
location

In [ ]:
## Plot figure
fig = pf.plot_behavior_across_days(direction_percentage, x_var = 'day', y_var = 'direction_percentage', avg_color = 'turquoise')
fig.update_layout(yaxis_title = 'Percent (%)')
fig.update_layout(title = {'text': 'Percentage of Time in Correct Direction',
                                   'xanchor': 'center',
                                   'y': 0.9,
                                   'x': 0.5})
fig.update_layout(width = 500, height = 500)
fig.add_hline(y = 50, line_dash = 'dash', line_width = 1, line_color = 'black')
fig.update_yaxes(range = [0, 80])
fig.show()
# fig.write_image('progress_figures/correct_direction.png')

In [ ]:
## Plot linearized position across a session
mouse = 'mc07'
date = '2022_10_03'
aligned_behavior = ctb.load_and_align_behavior(behavior_path, mouse, date, session = '30min', plot_frame_usage = False)

In [ ]:
## Plot
trials = ctb.get_trials(aligned_behavior, shift_factor = 0, angle_type = 'radians', counterclockwise = True)
forward_trials, reverse_trials = ctb.forward_reverse_trials(aligned_behavior, trials, positive_jump = 350, wiggle = 3)
fig = pf.plot_linearized_position(aligned_behavior, trials, forward_trials, reverse_trials, angle_type = 'radians', forward_trial_color = 'turquoise', reverse_trial_color = 'grey')
fig.update_layout(height = 500, width = 1500)
fig.show()
# fig.write_image('progress_figures/forward_reverse.png')

The code below is meant for dprime and correct rejection analysis on a trial by trial basis.

The below code block calculates all dprime metrics across all mice.

In [ ]:
## dprime across all mice and all sessions
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11']
bin_size = 5
mouse_signal_data = {}
mouse_agg_data = {}
mouse_signal_forward = {}
mouse_agg_forward = {}
for mouse in mouse_list:
    files = ctb.subset_combined(combined_list, mouse)
    signal_dict_all = {}
    signal_dict_forward = {}
    agg_dict_all = {}
    agg_dict_forward = {}
    for path in files:
        ## Extract date
        date = re.search('(2022_[0-9]+_[0-9]+)', behavior_path).group()
        ## Load circle track data
        circle_track = pd.read_csv(behavior_path)
        ## Get which ports are the reward ports
        reward_one, reward_two = ctb.get_rewarding_ports(circle_track)
        ## Normalize timestamps and crop data
        circle_track = ctb.normalize_timestamp(circle_track)
        circle_track = ctb.crop_data(circle_track)
        ## Align behavior data
        aligned_behavior = ctb.load_and_align_behavior(dpath, mouse, date, session = '30min', plot_frame_usage = False)
        ## Get timestamps where mouse licked (replaces REWARD with LICK)
        lick_tmp = ctb.get_licks(circle_track)
        ## Calculate trials, label each frame as a specific trial
        trials = ctb.get_trials(aligned_behavior, shift_factor = 0, angle_type = 'radians', counterclockwise = True)
        ## Calculate forward and reverse trials
        forward_trials, reverse_trials = ctb.forward_reverse_trials(aligned_behavior, trials, positive_jump = 350, wiggle = 3)
        ## Label each frame in aligned_behavior with a trial
        aligned_behavior.insert(1, 'trial', trials)
        ## Label lick data with what trial those licks occurred during
        lick_data = ctb.label_lick_trials(aligned_behavior, lick_tmp, trials)
        ## Calculate dprime, CR, FA, hits, misses for each trial
        signal_all = ctb.dprime_metrics(lick_data, trials, reward_one, reward_two)
        ## Calculate for only trials in the correct direction
        signal_forward = ctb.dprime_metrics(lick_data, forward_trials, reward_one, reward_two)
        ## Aggregate data every 5 trials
        agg_data = ctb.aggregate_metrics(signal_all, bin_size = bin_size)
        ## Aggregate data over forward trials
        agg_data_forward = ctb.aggregate_metrics(signal_forward, bin_size = bin_size)
        ## Append to dictionaries
        signal_dict_all[date] = signal_all
        signal_dict_forward[date] = signal_forward
        agg_dict_all[date] = agg_data
        agg_dict_forward[date] = agg_data_forward
    ## Append to mouse dictionaries
    mouse_signal_data[mouse] = signal_dict_all
    mouse_signal_forward[mouse] = signal_dict_forward
    mouse_agg_data[mouse] = agg_dict_all
    mouse_agg_forward[mouse] = mouse_agg_forward

In [ ]:
## Plot dprime values across all sessions
mouse_dprime = {}
for mouse in mouse_signal_forward:
    dprime_values = {}
    for date in mouse_signal_forward[mouse]:
        dprime_values[date] = np.nanmean(mouse_signal_forward[mouse][date]['dprime'])
    mouse_dprime[mouse] = dprime_values
## For mc03, the dates are in a strange order, so reorder the keys
desired_ordered_list = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_06', '2022_10_07']
mouse_dprime['mc03'] = {k: mouse_dprime['mc03'][k] for k in desired_ordered_list}
## Turn into a dataframe
dprime_df = pd.DataFrame()
for i, mouse in enumerate(mouse_dprime):
    data = pd.DataFrame(mouse_dprime[mouse], index = [i])
    dprime_df = pd.concat([dprime_df, data])
## Reorder columns
columns = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_05', '2022_10_06', '2022_10_07']
dprime_df = dprime_df[columns]
## Rename columns
dprime_df = dprime_df.rename(columns = {'2022_09_29': 'T1', '2022_09_30': 'T2', '2022_10_01': 'T3', '2022_10_02': 'T4', '2022_10_03': 'R1', '2022_10_04': 'R2', '2022_10_05': 'R3', '2022_10_06': 'R4', '2022_10_07': 'T_R'})
## Insert mice into df
dprime_df.insert(0, 'Mouse', mouse_list)
## Plot data
fig = pf.custom_graph_template(title = "d' Across Sessions", x_title = 'Session', y_title = "d'")
x_axis = ['T1', 'T2', 'T3', 'T4', 'R1', 'R2', 'R3', 'R4', 'T_R']
for row in dprime_df.index:
    plot_data = dprime_df.loc[row, 'T1':'T_R']
    fig.add_trace(go.Scatter(x = x_axis, y = plot_data, mode = 'lines', line_color = 'grey', line_width = 0.5, opacity = 0.5, name = dprime_df.loc[row, 'Mouse'], showlegend = False))
fig.add_trace(go.Scatter(x = x_axis, y = dprime_df.mean(axis = 0, numeric_only = True), error_y = dict(type = 'data', array = dprime_df.loc[:, 'T1':'T_R'].sem(axis = 0)), 
                         mode = 'lines + markers', line_color = 'turquoise', showlegend = False))
fig.show()
# fig.write_image('progress_figures/dprime_sessions.png')

In [ ]:
## Plot Correct Rejection Rate values across all sessions
mouse_cr = {}
for mouse in mouse_signal_forward:
    cr_values = {}
    for date in mouse_signal_forward[mouse]:
        cr_values[date] = np.nanmean(mouse_signal_forward[mouse][date]['CR'])
    mouse_cr[mouse] = cr_values
## For mc03, the dates are in a strange order, so reorder the keys
desired_ordered_list = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_06', '2022_10_07']
mouse_cr['mc03'] = {k: mouse_cr['mc03'][k] for k in desired_ordered_list}
## Turn into a dataframe
cr_df = pd.DataFrame()
for i, mouse in enumerate(mouse_cr):
    data = pd.DataFrame(mouse_cr[mouse], index = [i])
    cr_df = pd.concat([cr_df, data])
## Reorder columns
columns = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_05', '2022_10_06', '2022_10_07']
cr_df = cr_df[columns]
## Rename columns
cr_df = cr_df.rename(columns = {'2022_09_29': 'T1', '2022_09_30': 'T2', '2022_10_01': 'T3', '2022_10_02': 'T4', '2022_10_03': 'R1', '2022_10_04': 'R2', '2022_10_05': 'R3', '2022_10_06': 'R4', '2022_10_07': 'T_R'})
## Insert mice into df
cr_df.insert(0, 'Mouse', mouse_list)
## Plot data
fig = pf.custom_graph_template(title = 'Correct Rejection Rate', x_title = 'Session', y_title = 'Correct Rejection Rate')
x_axis = ['T1', 'T2', 'T3', 'T4', 'R1', 'R2', 'R3', 'R4', 'T_R']
for row in cr_df.index:
    plot_data = cr_df.loc[row, 'T1':'T_R']
    fig.add_trace(go.Scatter(x = x_axis, y = plot_data, mode = 'lines', line_color = 'grey', line_width = 0.5, opacity = 0.5, name = cr_df.loc[row, 'Mouse'], showlegend = False))
fig.add_trace(go.Scatter(x = x_axis, y = cr_df.mean(axis = 0, numeric_only = True), error_y = dict(type = 'data', array = cr_df.loc[:, 'T1':'T_R'].sem(axis = 0)), 
                         mode = 'lines + markers', line_color = 'turquoise', showlegend = False))
fig.add_hline(y = 1, line_dash = 'dash', line_color = 'black', line_width = 1, opacity = 0.5)
fig.show()
fig.write_image('progress_figures/correct_rejection.png')

In [ ]:
## Get distribution of trial times in the correct direction
time_diff_forward, time_diff_reverse = ctb.calculate_trial_length(aligned_behavior, forward_reverse = True)
fig = go.Figure()
fig.add_trace(go.Histogram(x = time_diff_forward, marker_color = 'grey'))
fig.update_layout(template = 'simple_white', height = 500, width = 500)
fig.update_xaxes(title = 'Trial Time (s)')
fig.update_yaxes(title = 'Count')
fig.update_layout(
    title={
        'text': 'Trial Time Distribution',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()
fig.write_image('progress_figures/trial_time_distribution.png')

In [ ]:
## Get average trial times for all mice
## Plot linearized position across a session
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11']
forward_reverse = True
angle_type = 'radians'

mouse_trial_times = {}
for mouse in mouse_list:
    ## Load mouse behavior
    mouse_behavior = ctb.import_mouse_behavior_data(behavior_path, mouse, key_file = 'keys.yml', session = '30min')
    if forward_reverse:
        trial_times_forward = {}
        trial_times_reverse = {}
    else:
        trial_times = {}
    for session in mouse_behavior:
        ## Calculate trials, label each frame as a specific trial
        trials = ctb.get_trials(mouse_behavior[session], shift_factor = 0, angle_type = angle_type, counterclockwise = True)
        if forward_reverse:
            ## Calculate forward and reverse trials
            forward_trials, reverse_trials = ctb.forward_reverse_trials(mouse_behavior[session], trials, positive_jump = 350, wiggle = 3)
            ## Get distribution of all trial times
            time_diff_forward, time_diff_reverse = ctb.calculate_trial_length(mouse_behavior[session], forward_reverse = True)
            ## Append to dict
            trial_times_forward[session] = time_diff_forward
            trial_times_reverse[session] = time_diff_reverse
        else:
            ## Get distribution of all trial times
            time_diff = ctb.calculate_trial_length(mouse_behavior[session], forward_reverse = False)
            ## Append to list
            trial_times[session] = time_diff
    if forward_reverse:
        mouse_trial_times[mouse] = {'forward': trial_times_forward, 'reverse': trial_times_reverse}
    else:
        mouse_trial_times[mouse] = trial_times

If looking at all trials, use the below cell block.

In [ ]:
## Create a plot of average trial time across each session for all mice
df = pd.DataFrame(mouse_trial_times).T
mouse_avg_trial_times = {}
for mouse in df.index:
    session_dict = {}
    for session in df.columns:
        trial_times = df.loc[mouse, session]
        ## Bin trial time to see if trials are longer earlier in the session
        bin_size = 2
        if type(trial_times) is not list:
            pass
        else:
            ## Create bins
            bins = np.arange(1, len(trial_times), bin_size)
            ## Bin data
            binned = np.split(trial_times, bins)
            ## Take the average of each bin
            avg = [np.mean(bin) for bin in binned]
            session_dict[session] = avg
    mouse_avg_trial_times[mouse] = session_dict
## Avg time across each bin
session_averages = ctb.trial_averages(mouse_avg_trial_times, session_list, forward = False)
avg_times = pd.DataFrame(mouse_avg_trial_times).T
avg_times = avg_times[session_list]
fig = make_subplots(rows = 3, cols = 3, shared_yaxes = 'all')
for mouse in avg_times.index:
    for i, session in enumerate(avg_times.columns):
        if (mouse == 'mc03') & (session == 'Reversal3'):
            pass
        else:
            data = avg_times.loc[mouse, session]
            x_axis = np.arange(0, len(data))
            if i <= 2:
                fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.7, showlegend = False), row = 1, col = i+1)
            elif (i > 2) & (i <= 5):
                fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.7, showlegend = False), row = 2, col = i-2)
            elif (i > 5) & (i <= 8):
                fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.7, showlegend = False), row = 3, col = i-5)
for i, session in enumerate(session_list):
    data = session_averages[session]
    x_axis = np.arange(0, len(data))
    if i <= 2:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', showlegend = False), row = 1, col = i+1)
    elif (i > 2) & (i <= 5):
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', showlegend = False), row = 2, col = i-2)
    else:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', showlegend = False), row = 3, col = i-5)
fig.update_layout(template = 'simple_white', width = 600, height = 600)
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Binned Trials', row = 3, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.update_layout(
    title={
        'text': 'Average Trial Time Across Session',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()
# fig.write_image('progress_figures/trial_time_all_cohort1.png')

If looking at forward trials only, use the below cell block.

In [ ]:
## Get average trial times for all mice
## Plot linearized position across a session
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11']
forward_reverse = True
angle_type = 'radians'

mouse_trial_times = {}
for mouse in mouse_list:
    ## Load mouse behavior
    mouse_behavior = ctb.import_mouse_behavior_data(behavior_path, mouse, key_file = 'keys.yml', session = '30min')
    if forward_reverse:
        trial_times_forward = {}
        trial_times_reverse = {}
    else:
        trial_times = {}
    for session in mouse_behavior:
        ## Calculate trials, label each frame as a specific trial
        trials = ctb.get_trials(mouse_behavior[session], shift_factor = 0, angle_type = angle_type, counterclockwise = True)
        if forward_reverse:
            ## Calculate forward and reverse trials
            forward_trials, reverse_trials = ctb.forward_reverse_trials(mouse_behavior[session], trials, positive_jump = 350, wiggle = 3)
            ## Get distribution of all trial times
            time_diff_forward, time_diff_reverse = ctb.calculate_trial_length(mouse_behavior[session], forward_reverse = True)
            ## Append to dict
            trial_times_forward[session] = time_diff_forward
            trial_times_reverse[session] = time_diff_reverse
        else:
            ## Get distribution of all trial times
            time_diff = ctb.calculate_trial_length(mouse_behavior[session], forward_reverse = False)
            ## Append to list
            trial_times[session] = time_diff
    if forward_reverse:
        mouse_trial_times[mouse] = {'forward': trial_times_forward, 'reverse': trial_times_reverse}
    else:
        mouse_trial_times[mouse] = trial_times

In [ ]:
mouse_forward_avg = {}
for mouse in mouse_trial_times:
    subset_dict = mouse_trial_times[mouse]['forward']
    session_dict = {}
    for session in subset_dict:
        trial_times = subset_dict[session]
        ## Bin trial time to see if trials are longer earlier in the session
        bin_size = 2
        if type(trial_times) is not list:
            pass
        else:
            ## Create bins
            bins = np.arange(1, len(trial_times), bin_size)
            ## Bin data
            binned = np.split(trial_times, bins)
            ## Take the average of each bin
            avg = [np.mean(bin) for bin in binned]
            session_dict[session] = avg
    mouse_forward_avg[mouse] = session_dict
## Avg time across each bin
session_averages = ctb.trial_averages(mouse_forward_avg, session_list, forward = True)
avg_times = pd.DataFrame(mouse_forward_avg).T
avg_times = avg_times[session_list]
fig = make_subplots(rows = 3, cols = 3,
                    subplot_titles = session_list,
                    shared_yaxes = 'all')
for mouse in avg_times.index:
    for i, session in enumerate(avg_times.columns):
        if (mouse == 'mc03') & (session == 'Reversal3'):
            pass
        else:
            data = avg_times.loc[mouse, session]
            x_axis = np.arange(0, len(data))
            if i <= 2:
                fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.5, showlegend = False), row = 1, col = i+1)
            elif (i > 2) & (i <= 5):
                fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.5, showlegend = False), row = 2, col = i-2)
            elif (i > 5) & (i <= 8):
                fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.5, showlegend = False), row = 3, col = i-5)
for i, session in enumerate(session_list):
    data = session_averages[session]
    x_axis = np.arange(0, len(data))
    if i <= 2:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', showlegend = False), row = 1, col = i+1)
    elif (i > 2) & (i <= 5):
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', showlegend = False), row = 2, col = i-2)
    else:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', showlegend = False), row = 3, col = i-5)
fig.update_layout(template = 'simple_white', width = 600, height = 600)
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Trial', row = 3, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.update_layout(
    title={
        'text': 'Binned Forward Trial Time Across Session',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()
# fig.write_image('progress_figures/trial_time_forward_cohort1.png')

Create a dataframe with number of trials, trial length, max activation strength, mean activation strength

In [6]:
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09']
angle_type = 'radians'
mouse_results = {}
for mouse in mouse_list:
    mouse_behavior = ctb.import_mouse_behavior_data(behavior_path, mouse, key_file = 'minian_keys.yml', session = '30min')
    spath = pjoin(ensemble_path, mouse)
    session_dictionary = {}
    for session in mouse_behavior:
        data = mouse_behavior[session]
        ## Load assemblies
        assemblies = ica.load_session_assemblies(mouse, spath = spath, session_id = session)
        act = assemblies['activations'].values
        aligned_behavior = ica.align_activations_to_behavior(act, data)
        trials = ctb.get_trials(aligned_behavior, shift_factor = 0, angle_type = angle_type, counterclockwise = True)
        aligned_behavior.insert(1, 'trial', trials)
        ## Calculate forward and reverse trials
        forward_trials, reverse_trials = ctb.forward_reverse_trials(aligned_behavior, trials, positive_jump = 350, wiggle = 3)
        results = {'trial': [], 'trial_length': [], 'max': [], 'mean': []}
        for trial in forward_trials:
            behavior = aligned_behavior[aligned_behavior.trial == trial]
            first, last = behavior.timestamp.to_numpy()[0], behavior.timestamp.to_numpy()[-1]
            trial_length = last - first
            ## Get max and mean of activation strength within a trial
            trial_activation = act[:, aligned_behavior.trial == trial]
            results['max'].append(np.mean(np.nanmax(trial_activation, axis = 1)))
            results['mean'].append(np.mean(np.mean(trial_activation, axis = 1)))
            ## Append trial number and trial length
            results['trial'].append(trial)
            results['trial_length'].append(trial_length)
        session_dictionary[session] = results
    mouse_results[mouse] = session_dictionary   
## Create dataframe
trial_df = pd.DataFrame()
for mouse in mouse_list:
    for session in session_dict[mouse]:
        df = pd.DataFrame(mouse_results[mouse][session])
        df.insert(4, 'mouse', mouse)
        df.insert(5, 'session', session)
        trial_df = pd.concat([trial_df, df], ignore_index = True)

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/EnsembleRemodeling_Resubmission/../circletrack_behavior.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.timestamp = (pd.to_numeric(data.timestamp) - start_time)
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/EnsembleRemodeling_Resubmission/../circletrack_behavior.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.timestamp = (pd.to_numeric(data.timestamp) - start_time)
/media/caishuman/csstorage3/Austin

In [18]:
## Linear regression
lm_max = pg.linear_regression(trial_df.trial_length, trial_df['max'], as_dataframe = False)
lm_mean = pg.linear_regression(trial_df.trial_length, trial_df['mean'], as_dataframe = False)
## Plot max
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Max Activation Strength')
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = trial_df['max'], mode = 'markers', opacity = 0.7, showlegend = False, 
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = lm_max['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 250, y = 50, text = 'R<sup>2</sup> = {}'.format(round(lm_max['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 247, y = 20, text = 'p = 2.884e-284', showarrow = False)
fig.show()
fig.write_image(pjoin(figure_path, 'trial_length_vs_activation_allsessions.png'))

In [ ]:
## Linear regression
lm_trial = pg.linear_regression(trial_df.trial, trial_df.trial_length, as_dataframe = False)
## Plot max
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial', y_title = 'Trial Length')
fig.add_trace(go.Scatter(x = trial_df.trial, y = trial_df.trial_length, mode = 'markers', opacity = 0.7, showlegend = False, 
                         marker={'color': trial_df['max'], 'cmin': 0, 'cmax': 300, 'showscale': True, 'line': {'color': 'black', 'width' : 0.5}}))
# fig.add_trace(go.Scatter(x = trial_df.trial, y = lm_trial['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.show()

In [11]:
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Mean Activation Strength')
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = trial_df['mean'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = lm_mean['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.show()

In [20]:
## Linear regression FOR JUST REVERSAL1
reversal_df = trial_df[trial_df.session == 'Reversal1']
lm_max_reversal = pg.linear_regression(reversal_df.trial_length, reversal_df['max'], as_dataframe = False)
lm_mean_reversal = pg.linear_regression(reversal_df.trial_length, reversal_df['mean'], as_dataframe = False)
fig = pf.custom_graph_template('Reversal1', x_title = 'Trial Length (s)', y_title = 'Max Activation Strength')
fig.add_trace(go.Scatter(x = reversal_df.trial_length, y = reversal_df['max'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = reversal_df.trial_length, y = lm_max_reversal['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 200, y = 50, text = 'R<sup>2</sup> = {}'.format(round(lm_max_reversal['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 197, y = 20, text = 'p = 1.632e-57', showarrow = False)
fig.show()
fig.write_image(pjoin(figure_path, 'trial_length_vs_max_reversal1.png'))

In [19]:
lm_max_reversal

{'names': ['Intercept', 'trial_length'],
 'coef': array([8.53370561, 2.02679645]),
 'se': array([2.46998732, 0.10075151]),
 'T': array([ 3.45495928, 20.11678548]),
 'pval': array([6.30013208e-04, 1.63152867e-57]),
 'r2': 0.5750940046897388,
 'adj_r2': 0.5736729144044201,
 'CI[2.5%]': array([3.67294428, 1.82852457]),
 'CI[97.5%]': array([13.39446693,  2.22506832]),
 'df_model': 1,
 'df_resid': 299,
 'residuals': array([-1.92766193e+01, -8.85081087e+00, -1.29638081e+01, -3.42799814e+01,
         1.66001825e+01,  7.72826407e+01,  7.88341899e+01,  2.30196152e+01,
         3.70331002e+00,  2.52838072e+01,  3.57538613e+01,  4.81084268e+01,
         2.54190222e+01,  1.20896249e+01, -1.34518908e+01,  3.29912105e+01,
        -4.18574784e-01,  1.71422610e+01,  1.51518563e+01,  8.50438234e+00,
         3.24328717e+01,  2.49935244e+01, -1.34402016e+01,  1.57154961e+01,
        -4.89758084e+00,  1.30358735e+01,  2.16743821e+01, -1.46003231e+01,
        -1.23817408e+01, -1.83758416e+01, -6.67997225e

The code below is to plot the peak correct rejection rate against the proportion of fading ensembles.

In [ ]:
## Set mouse list, path, and z_thresh for binarization
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09'] ## excluded mc11 from the analysis because three sessions had to be stitched together
across_time = False
x_bin_size = 15
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
## Loop through each mouse
for mouse in tqdm(mouse_list):
    ## saved ensemble data path
    spath = pjoin(ensemble_path, mouse)
    ## load in behavior data across all sessions
    if not across_time:
        mouse_behavior = ctb.import_mouse_behavior_data(behavior_path, mouse, key_file = 'keys.yml', session = '30min')
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    ## Load assemblies
    for session in session_dict[mouse]:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, spath = spath, format = 'pickle', session_id = session)
        ## Load a specific session's behavior data
        if not across_time:
            aligned_behavior = mouse_behavior[session]
            ## Ensure that activations and aligned behavior are the same length
            aligned_behavior = ica.align_activations_to_behavior(assemblies['activations'], aligned_behavior)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = ctb.get_trials(aligned_behavior, shift_factor = 0, angle_type = 'radians', counterclockwise = True)
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(assemblies['activations'], z_threshold = None, x_bin_size = x_bin_size, analysis_type = 'max', zscored = True, alpha = 'sidak')  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(assemblies['activations'], aligned_behavior, trials, trial_type = 'forward', z_threshold = None)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends

In [ ]:
## Plot Correct Rejection Rate values across all sessions
mouse_cr_max = {}
for mouse in mouse_signal_forward:
    cr_max = {}
    for date in mouse_signal_forward[mouse]:
        cr_values[date] = np.max(mouse_signal_forward[mouse][date]['CR'])
    mouse_cr_max[mouse] = cr_max
## For mc03, the dates are in a strange order, so reorder the keys
desired_ordered_list = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_06', '2022_10_07']
mouse_cr_max['mc03'] = {k: mouse_cr_max['mc03'][k] for k in desired_ordered_list}
# ## Turn into a dataframe
# cr_df = pd.DataFrame()
# for i, mouse in enumerate(mouse_cr):
#     data = pd.DataFrame(mouse_cr[mouse], index = [i])
#     cr_df = pd.concat([cr_df, data])
# ## Reorder columns
# columns = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_05', '2022_10_06', '2022_10_07']
# cr_df = cr_df[columns]
# ## Rename columns
# cr_df = cr_df.rename(columns = {'2022_09_29': 'T1', '2022_09_30': 'T2', '2022_10_01': 'T3', '2022_10_02': 'T4', '2022_10_03': 'R1', '2022_10_04': 'R2', '2022_10_05': 'R3', '2022_10_06': 'R4', '2022_10_07': 'T_R'})
# ## Insert mice into df
# cr_df.insert(0, 'Mouse', mouse_list)
